1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 질문과 같이 전달

1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters langchain-chroma pypdf

2. Knowledge Base 구성을 위한 데이터 생성

- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
  - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
  - 비용 감소와 답변 생성시간 감소의 효과
  - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = PyPDFLoader('./start-up_investor_relations.pdf')  # PDF 파일 경로를 지정
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)
document_list[1].page_content[:1000]

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(
    model="embedding-query"
)

In [8]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때
# database = Chroma(collection_name='chroma', persist_directory="./chroma", embedding_function=embedding)

3. 답변 생성을 위한 Retrieval

Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [15]:
query = '스타트업 시작시 투자자에게 보여줘야 할 자료는?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=7)

In [21]:
retrieved_docs[0].page_content[:1000]

'186 187D 단계적인 로드맵을 보여줘라 \n로마는 하루아침에 이루어지지 않았듯이, 스타트업은 각 단계를 거\n쳐 성장하게 됩니다. 미래의 비전을 이야기하되 현재의 단계에서 검\n증된 가설과 지표는 어떠하며, 다음 단계로 가기 위해 검증해야 할 가\n설과 지표, 그리고 이를 달성하기 위한 중기적인 전략은 무엇인지가 \n구체적이어야 하죠. 먼 미래의 추가적인 수익 모델이나 사업 확장은, \n아직은 요원하기 때문에 투자자에게는 일종의 보너스 정도로만 느껴\n질 수 있습니다.\n \nE 적절한 스토리텔링을 활용하라 \n창업 동기나 시장의 현황과 배경, 고객 문제 등에 대해 적절한 스토\n리텔링이 있으면 좋습니다. 사람은 감성적인 동물이기도 하기에 논리\n적인 개연성 외에 감동도 있으면 설득과 공감에 유리합니다.\n그리고 예를 들어 단순히 우리의 제품/서비스가 좋다고 주장하기보\n다는 구체적인 사례와 함께 고객의 평점이나 반응/리뷰 등을 서술해 \n주는 것이 더 현실적으로 느껴져서 효과적입니다.\nF 텍스트와 시각 자료를 조화롭게 사용하라 \n너무 텍스트 위주이거나 반대로 너무 많은 도식과 표를 동원하면 \n투자자 입장에서 이해하기 힘들 수 있습니다. 따라서 적절한 레이아웃을 취해야 하며, 폰트 종류나 크기도 적절한 것을 활용하는 게 가독\n성에 좋죠. 예쁘게 만들기 이전에 먼저 명확하게 만들어야 합니다. 그\n리고 필요시 이미지도 적절하게 이용하면 좋고, 특히 기술적으로 이\n해가 어려운 부분은 데모 동영상36이나 스크린샷 등을 활용하면 상대\n적으로 쉽게 투자자를 이해시킬 수도 있습니다. \ncase\n한 테크 스타트업의 경우 기술적인 부분이 어려워 설명하기가 쉽지 않았습니다. 그래서 3D \n동영상이 아니더라도 간단한 2D 동영상으로 구현해 보면 어떻겠냐고 자문하였는데, 실제로 \n투자자가 쉽게 이해가 가능해진 적도 있죠.\nG 어깨에 힘을 빼고 본인 스타일대로 자료를 만들어라 \n자료를 포장하는 것에만 너무 신경 쓰지 말고, 어깨에 힘을 빼고 스\n스로 솔

4. Augmentation을 위한 Prompt 활용

Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [22]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model='solar-pro')

In [23]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/crazyatom/.pyenv/versions/3.11.11/envs/start-up-bot/lib/python3.11/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
  - RetrievalQA는 create_retrieval_chain으로 대체됨
  - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [24]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [25]:
# ai_message = qa_chain({"query": query})

# 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [26]:
ai_message

{'query': '스타트업 시작시 투자자에게 보여줘야 할 자료는?',
 'result': '재무(Financial): 3년 이내의 매출 계획과 예상되는 비용을 포함합니다. 초기 스타트업의 경우 3년 이내의 매출 계획, 필요한 투자금액과 투자금을 어떻게 활용할지, 그리고 자금 소진 속도(Burn Rate) 정도가 필요합니다.'}